In [61]:
import pandas as pd
import numpy as np
import networkx as nx

path = "dataset/titanic/"
dt_train = pd.read_csv(path+"train.csv", usecols=['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']).dropna()

dt_train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


In [62]:
# Transformando os dados de texto para dados numéricos e inteiros:

dt_train.replace(['male', 'female'], [0,1], inplace=True)
dt_train['Embarked'].replace(['S', 'C', 'Q'], [1,2,3], inplace=True)
freq = frequencyList(dt_train['Age'])

In [63]:
dt_train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,0,22.0,7.2500,1
1,1,1,1,38.0,71.2833,2
2,1,3,1,26.0,7.9250,1
3,1,1,1,35.0,53.1000,1
4,0,3,0,35.0,8.0500,1


In [64]:
def classInterval(dataFrame):
    numClasses = 1 + np.log2(dataFrame.count())
    return int((dataFrame.max()-dataFrame.min())/numClasses)

def frequencyList(dataFrame):
    freqList = []
    interval = classInterval(dataFrame)
    for tam in range(int(dataFrame.min()), int(dataFrame.max()), interval):
        freqList.append(tam)
    return freqList

def isContinuous(dataFrame):
    if np.size(dataFrame.unique()) >= 10:
        return True
    else:
        return False

def finalEntropy(entropies, total):
    entropy = 0
    for t, e in zip(total, entropies):
        if np.isnan(e):
            e = 0
        entropy = entropy + t/sum(total)*e
    return entropy

def innerFunc(denominator, survived, notSurvived):
    if denominator == 0 or survived == 0 or notSurvived == 0:
        return 0
    return (-(notSurvived/denominator*np.log2(notSurvived/denominator))-(survived/denominator*np.log2(survived/denominator)))

def gain_calculator(dt_train, targClass_name):
    s_total = []
    entropies = []
    gainList = []
    temp = 0
    targClass = dt_train[targClass_name]
    targEntropy = targetClassEntropy(dt_train[targClass_name])
    for class_ in dt_train:
        if not(class_ == targClass_name):
            dt_caracteristic = dt_train[class_]
            if isContinuous(dt_caracteristic):
                intervalList = frequencyList(dt_caracteristic)
                for inter in intervalList:
                    notSurvived = dt_train[(dt_caracteristic>temp) & (dt_caracteristic<=inter) & (targClass==0)].count()[0]
                    survived = dt_train[(dt_caracteristic>temp) & (dt_caracteristic<=inter) & (targClass==1)].count()[0]
                    temp = inter
                    total = notSurvived+survived
                    s_total.append(total)
                    entropies.append(innerFunc(total, survived, notSurvived))
            else:
                for p in dt_caracteristic.unique():
                    notSurvived = dt_train[(dt_caracteristic==p) & (targClass==0)].count()[0]
                    survived = dt_train[(dt_caracteristic==p) & (targClass==1)].count()[0]
                    total = notSurvived+survived
                    s_total.append(total)
                    entropies.append(innerFunc(total, survived, notSurvived))
            classEntropy = finalEntropy(entropies, s_total)
            gainList.append([class_, (targEntropy - classEntropy)])
    return gainList

def gain_calculator2(dt_train, targClass_name, interval=None, attribut=None):
    s_total = []
    entropies = []
    gainList = []
    temp = 0
    targClass = dt_train[targClass_name]
    targEntropy = targetClassEntropy(dt_train[targClass_name])
    for class_ in dt_train:
        dt_caracteristic = dt_train[class_]
        if isContinuous(dt_caracteristic):
            notSurvived = dt_train[(dt_caracteristic>interval[0]) & (dt_caracteristic<=interval[1]) & (targClass==0)].count()[0]
            survived = dt_train[(dt_caracteristic>interval[0]) & (dt_caracteristic<=interval[1]) & (targClass==1)].count()[0]
            total = notSurvived+survived
            s_total.append(total)
            entropies.append(innerFunc(total, survived, notSurvived))
        else:
            notSurvived = dt_train[(dt_caracteristic==attribut) & (targClass==0)].count()[0]
            survived = dt_train[(dt_caracteristic==attribut) & (targClass==1)].count()[0]
            total = notSurvived+survived
            s_total.append(total)
            entropies.append(innerFunc(total, survived, notSurvived))
        classEntropy = finalEntropy(entropies, s_total)
        gainList.append([class_, (targEntropy - classEntropy)])
    return gainList

def targetClassEntropy(dt_class):
    s_total = []
    entropies = []
    for p in dt_class.unique():
        notSurvived = dt_train[dt_class==0].count()[0]
        survived = dt_train[dt_class==1].count()[0]
        total = notSurvived+survived
        s_total.append(total)
        entropies.append(innerFunc(total, survived, notSurvived))
    return finalEntropy(entropies, s_total)

In [66]:
def verificar(dataFrame, targClass):
    if np.size(dataFrame[targClass].unique()) == 1:
        return True
    else:
        return False
    
def bestGain(attributs):
    best = 0
    index = 0
    for i in range(attributs.size):
        if best < attributs[i][1]:
            best = attributs[i][1]
            index = i
    return index

def geraNo(dataFrame, atributo, targClass_name, no_atual, no_value):
    verif = verificar(dataFrame, targClass_name)
    if verif:
        no_value += 1
        v = survived.unique().item()
        g.add_node(no_value, result=v)
        g.add_edge(no_atual, no_value, condition=atributo)
    if dataFrame.columns.size > 1:
        gain = gain_calculator(dataFrame, targClass_name)
        index = bestGain(gain)
        #####
        if atributos > 0:
            pass
        else:
            pass
    else:
        #Cria folha com a classe
        if dataFrame[targClass==1].count() > dataFrame[targClass==0].count():
            dataFrame = dataFrame[targClass==1]
            if isContinuous(dataFrame):
                newNode = Node(dataFrame.name, dataFrame.value.min(), 1, isContinous=True)
            else:
                tam = 0
                condition = None
                for at in dataFrame.unique():
                    if dataFrame[dataFrame==at].count() > tam:
                        tam = dataFrame[dataFrame==at].count()
                        condition = at
                newNode = Node(dataFrame.name, condition, 1, isContinous=False)
        else:
            dataFrame = dataFrame[targClass==0]
            if isContinuous(dataFrame):
                newNode = Node(dataFrame.name, dataFrame.value.min(), 0, isContinous=True)
            else:
                tam = 0
                condition = None
                for at in dataFrame.unique():
                    if dataFrame[dataFrame==at].count() > tam:
                        tam = dataFrame[dataFrame==at].count()
                        condition = at
                newNode = Node(dataFrame.name, condition, 0, isContinous=False)
    
    

In [67]:
gain = gain_calculator(dt_train, 'Survived')
gain

[['Pclass', 0.09400998456880616],
 ['Sex', 0.15405914870226456],
 ['Age', 0.11380147316047828],
 ['Fare', 0.10127070898784996],
 ['Embarked', 0.08654270068465386]]